In [38]:
# Basics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots


# Tools
from copy import copy # Shallow copy
from itertools import product
from collections import defaultdict
from functools import partial
from IPython.display import display # Allows functions to simultaneously return values and show tables

# Styling
from colorama import Fore
from colorama import Style
from matplotlib.colors import Colormap


# Assessing Feature Importance
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import mutual_info_classif

# Pipeline
from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

# t-SNE
from sklearn.manifold import TSNE


# Dendogram
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform


# Kde Plots
from scipy.stats import gaussian_kde



# Probability plots
from scipy.stats import probplot

# The Tree Trio
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


import scipy.stats as stats


from utils import *

# Sequential Feature Selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [3]:
import pandas as pd
train = pd.read_csv(r"C:\Users\Nebula PC\Desktop\Projects\Academic-Success-Prediction\data\train.csv", index_col="id").rename(columns=str.strip)
test = pd.read_csv(r"C:\Users\Nebula PC\Desktop\Projects\Academic-Success-Prediction\data\test.csv", index_col="id").rename(columns=str.strip)

target = "Target"

value_mapping = {
    'Enrolled': 2,
    'Dropout': 0,
    'Graduate': 1
}

# Replace the values in the "Target" column
train['Target'] = train['Target'].replace(value_mapping)


In [4]:
# Calculate value counts
value_counts = train['Target'].value_counts().sort_index()

# Create Plotly bar chart
fig = go.Figure(data=[
    go.Bar(x=value_counts.index.astype(str), y=value_counts.values, marker_color=[ORANGE, TEAL, DARK_TEAL])
])

# Update layout with style parameters
fig.update_layout(
    title='Value Counts of Target Column',
    xaxis=dict(title='Target Categories', tickfont=dict(color=FONT_COLOR)),
    yaxis=dict(title='Count', tickfont=dict(color=FONT_COLOR)),
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    font=dict(color=FONT_COLOR),
    width = 1000,
    height = 1000
    
)

# Show the plot
fig.show()

In [6]:

for column in train.columns:
    print(f"{len(train[column].value_counts())} unique values in {column}")

6 unique values in Marital status
22 unique values in Application mode
8 unique values in Application order
19 unique values in Course
2 unique values in Daytime/evening attendance
21 unique values in Previous qualification
110 unique values in Previous qualification (grade)
18 unique values in Nacionality
35 unique values in Mother's qualification
39 unique values in Father's qualification
40 unique values in Mother's occupation
56 unique values in Father's occupation
668 unique values in Admission grade
2 unique values in Displaced
2 unique values in Educational special needs
2 unique values in Debtor
2 unique values in Tuition fees up to date
2 unique values in Gender
2 unique values in Scholarship holder
46 unique values in Age at enrollment
2 unique values in International
21 unique values in Curricular units 1st sem (credited)
24 unique values in Curricular units 1st sem (enrolled)
36 unique values in Curricular units 1st sem (evaluations)
23 unique values in Curricular units 1st

In [7]:
binary_columns = []

for column in train.columns:
    if len(train[column].value_counts()) == 2:
        binary_columns.append(column)

binary_columns

['Daytime/evening attendance',
 'Displaced',
 'Educational special needs',
 'Debtor',
 'Tuition fees up to date',
 'Gender',
 'Scholarship holder',
 'International']

In [8]:
# Check that all the binary columns are of the form 0 and 1
train[binary_columns].describe().T[['min', 'max']].style.set_table_styles(DF_STYLE)
#perfect

,min,max
Daytime/evening attendance,0.000000,1.000000
Displaced,0.000000,1.000000
Educational special needs,0.000000,1.000000
Debtor,0.000000,1.000000
Tuition fees up to date,0.000000,1.000000
Gender,0.000000,1.000000
Scholarship holder,0.000000,1.000000
International,0.000000,1.000000


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76518 entries, 0 to 76517
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  76518 non-null  int64  
 1   Application mode                                76518 non-null  int64  
 2   Application order                               76518 non-null  int64  
 3   Course                                          76518 non-null  int64  
 4   Daytime/evening attendance                      76518 non-null  int64  
 5   Previous qualification                          76518 non-null  int64  
 6   Previous qualification (grade)                  76518 non-null  float64
 7   Nacionality                                     76518 non-null  int64  
 8   Mother's qualification                          76518 non-null  int64  
 9   Father's qualification                      

In [11]:
#train.Target

In [12]:
#train.info()

In [13]:
missing_values_cols = train.isna().sum()[train.isna().sum() > 0].index.to_list()

print("Training Dataset Missing Values\n")

for feature in missing_values_cols:
    print(
        (feature) + "\t",
        (str(train[feature].isna().sum())) + "\t",
        (f"{train[feature].isna().sum() / len(train):.1%}") + "\t",
        (f"{train[feature].dtype}")
    )

Training Dataset Missing Values



In [14]:
print(f"Number of duplicated rows: {train.drop(columns = target).duplicated().sum()}")

Number of duplicated rows: 0


In [15]:
#numeric_description(train, target, styling = True, gradient = True, float_precision = 3, gradient_axis = 1)

In [16]:
pearson_corr, lower_tri_corr = correlation_values(train, target, 15, 1500, 1500)

In [ ]:
#dendogram(train, target, pearson_corr)

In [17]:
def get_kde_estimation(data_series):
    kde = gaussian_kde(data_series.dropna(), bw_method= None)
    kde_range = np.linspace(
        data_series.min() - data_series.max() * 0.1,
        data_series.max() + data_series.max() * 0.1,
        len(data_series),
    )
    estimated_values = kde.evaluate(kde_range)
    estimated_values_cum = np.cumsum(estimated_values)
    estimated_values_cum /= estimated_values_cum.max()
    return kde_range, estimated_values, estimated_values_cum


def get_n_rows_axes(n_features, n_cols=5, n_rows=None):
    n_rows = int(np.ceil(n_features / n_cols))
    current_col = range(1, n_cols + 1)
    current_row = range(1, n_rows + 1)
    return n_rows, list(product(current_row, current_col))

In [18]:
def high_corr_combinations(correlation_threshold, lower_triangular_correlations):
    corr_threshold = correlation_threshold
    lower_triangular_corr = lower_triangular_correlations

    highest_abs_corr = (
        lower_triangular_corr.abs()
        .unstack()
        .sort_values(ascending=False)  # type: ignore
        .rename("Absolute Pearson Correlation")
    )

    highest_abs_corr = (
        highest_abs_corr[highest_abs_corr > corr_threshold]
        .to_frame()
        .reset_index(names=["Feature 1", "Feature 2"])
    )

    highest_corr_combinations = highest_abs_corr[["Feature 1", "Feature 2"]].to_numpy()
    display(highest_abs_corr.style.set_table_styles(DF_STYLE).format(precision=2))
    return highest_corr_combinations



high_corr_combos = high_corr_combinations(0.89, lower_tri_corr)

,Feature 1,Feature 2,Absolute Pearson Correlation
0,Curricular units 1st sem (enrolled),Curricular units 2nd sem (enrolled),0.96
1,Curricular units 1st sem (credited),Curricular units 2nd sem (credited),0.93
2,Curricular units 1st sem (approved),Curricular units 2nd sem (approved),0.92
3,Mother's occupation,Father's occupation,0.90


In [ ]:
train.Target.values

In [ ]:
# Scatter

n_cols = 2
n_rows, axes = get_n_rows_axes(len(high_corr_combos), n_cols=n_cols)

fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    horizontal_spacing=0.1,
    vertical_spacing=0.02,
)

show_legend = True

for k, ((current_row, current_col), (feature1, feature2)) in enumerate(
    zip(axes, high_corr_combos)
):
    if k > 0:
        show_legend = False

    fig.add_scatter(
        x=train.query("Target == 'Graduate'")[feature1],
        y=train.query("Target == 'Graduate'")[feature2],
        mode="markers",
        name="Graduate",
        marker=dict(color = TEAL, size=5, symbol="diamond", opacity=0.7),
        legendgroup="Graduate",
        showlegend=show_legend,
        row=current_row,
        col=current_col,
    )
    fig.add_scatter(
        x=train.query("Target == 'Dropout'")[feature1],
        y=train.query("Target == 'Dropout'")[feature2],
        mode="markers",
        name="Dropout",
        marker=dict(color= ORANGE, size=5, symbol="circle", opacity=0.7),
        legendgroup="Dropout",
        showlegend=show_legend,
        row=current_row,
        col=current_col,
    )
    fig.add_scatter(
        x=train.query("Target == 'Enrolled'")[feature1],
        y=train.query("Target == 'Enrolled'")[feature2],
        mode="markers",
        name="Enrolled",
        marker=dict(color= DARKEST_TEAL, size=5, symbol="circle", opacity=0.7),
        legendgroup="Enrolled",
        showlegend=show_legend,
        row=current_row,
        col=current_col,
    )






    fig.update_xaxes(
        #type="log",
        title_text=feature1,
        titlefont_size=9,
        titlefont_family="Arial Black",
        tickfont_size=7,
        row=current_row,
        col=current_col,
    )
    fig.update_yaxes(
        #type="log",
        title_text=feature2,
        titlefont_size=9,
        titlefont_family="Arial Black",
        tickfont_size=7,
        row=current_row,
        col=current_col,
    )

fig.update_annotations(font_size=14)
fig.update_layout(
    font_color=FONT_COLOR,
    title="Highest Pearson Correlations - Pair Plots<br>Double Logarithmic Scale",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    width=1000,
    height=2000,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        xanchor="right",
        y=1.01,
        x=1,
        itemsizing="constant",
    ),
)

fig.show()

In [24]:
train_sampled = train.sample(10000)

In [25]:
numeric_data = train_sampled.select_dtypes("number")
numeric_cols = numeric_data.drop("Target", axis=1).columns.tolist()
n_cols = 5
n_rows, axes = get_n_rows_axes(len(numeric_cols))

In [ ]:
fig1 = make_subplots(
    rows=n_rows,
    cols=n_cols,
    y_title="Probability Density",
    horizontal_spacing=0.06,
    vertical_spacing=0.04,
)
fig2 = copy(fig1)

show_legend = True

for k, ((current_row, current_col), feature) in enumerate(zip(axes, numeric_cols)):
    if k > 0:
        show_legend = False

    for target, color in zip((0, 1, 2), (TEAL, ORANGE, DARKEST_TEAL)):
        kde_range, estimated_values, estimated_values_cum = get_kde_estimation(
            numeric_data.query(f"Target == {target}")[feature]
        )

        for fig, kde_values in zip(  # type: ignore
            (fig1, fig2), (estimated_values, estimated_values_cum)
        ):
            fig.add_scatter(
                x=kde_range,
                y=kde_values,
                line=dict(dash="solid", color=color, width=1),
                fill="tozeroy",
                name=f"Class {target}",
                legendgroup=f"Class {target}",
                showlegend=show_legend,
                row=current_row,
                col=current_col,
            )
            fig.update_yaxes(
                tickfont_size=7,
                row=current_row,
                col=current_col,
            )
            fig.update_xaxes(
                title_text=feature,
                titlefont_size=9,
                titlefont_family="Arial Black",
                tickfont_size=7,
                row=current_row,
                col=current_col,
            )

title1 = "Numerical Features - Kernel Density Estimation"
title2 = "Numerical Features - Cumulative Kernel Density Estimation"

for fig, title in zip((fig1, fig2), (title1, title2)):
    fig.update_annotations(font_size=14)
    fig.update_layout(
        font_color=FONT_COLOR,
        title=title,
        title_font_size=18,
        plot_bgcolor=BACKGROUND_COLOR,
        paper_bgcolor=BACKGROUND_COLOR,
        width=2000,
        height=3500,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            xanchor="right",
            y=1.01,
            x=1,
        ),
    )

fig1.show()

In [ ]:
fig2.show()

In [ ]:
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    y_title="Observed Values",
    x_title="Theoretical Quantiles",
    horizontal_spacing=0.06,
    vertical_spacing=0.04,
)
fig.update_annotations(font_size=14)

for (row, col), feature in zip(axes, numeric_cols):
    (osm, osr), (slope, intercept, R) = probplot(train_sampled[feature].dropna(), rvalue=True)
    x_theory = np.array([osm[0], osm[-1]])
    y_theory = intercept + slope * x_theory
    R2 = f"R\u00b2 = {R * R:.2f}"
    fig.add_scatter(x=osm, y=osr, mode="markers", row=row, col=col, name=feature)
    fig.add_scatter(x=x_theory, y=y_theory, mode="lines", row=row, col=col)
    fig.add_annotation(
        x=-1.25,
        y=osr[-1] * 0.75,
        text=R2,
        showarrow=False,
        row=row,
        col=col,
        font_size=9,
    )
    fig.update_yaxes(tickfont_size=7, row=row, col=col)
    fig.update_xaxes(
        title_text=feature,
        titlefont_size=9,
        titlefont_family="Arial Black",
        tickfont_size=7,
        row=row,
        col=col,
    )

fig.update_layout(
    font_color=FONT_COLOR,
    title="Numerical Features - Probability Plots against Normal Distribution",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    showlegend=False,
    width=2000,
    height=3500,
)
fig.update_traces(
    marker=dict(size=1, symbol="x-thin", line=dict(width=2, color=DARK_TEAL)),
    line_color=ORANGE,
)
fig.show()

need to drop gdp and inflation rate, will make it easier since they have negative values and they alreaady seem to be distributed relatively normally


Log Transformation - generally works fine with right-skewed data. Requires non-negative numbers.

Square Root Transformation - similarly to log-level transformation. Requires non-negative numbers.

Square Transformation - helps to reduce left-skewed data.

Reciprocal Transformation - used sometimes, when data is skewed, or there are obvious outliers. Not defined at zero.

Box-Cox Transformation - used when data is skewed or has outliers. Requires strictly positive numbers.

Yeo-Johnson Transformation - variation of Box-Cox transformation, but without restrictions concerning numbers.

In [26]:
#remove binary columns before doing transformations
numeric_cols = [col for col in numeric_cols if col not in binary_columns]

In [27]:
positive_features = list(train_sampled[numeric_cols].describe().T.query("min > 0").index)
zero_features = list(train_sampled[numeric_cols].describe().T.query("min == 0").index)
negative_features = list(train_sampled[numeric_cols].describe().T.query("min < 0").index)

In [28]:
epsilon = 0.0001
GDP_shift = 4.0601
Inflation_shift = 0.8001
r2_scores = defaultdict(tuple)

for feature in numeric_cols:
    orig = train_sampled[feature].dropna()
    if feature in positive_features:
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log) = probplot(np.log(orig), rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig), rvalue=True)
        _, (*_, R_sqrt) = probplot(np.sqrt(orig), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig), rvalue=True)
        _, (*_, R_reci) = probplot(np.reciprocal(orig), rvalue=True)
        _, (*_, R_boxcox) = probplot(stats.boxcox(orig)[0], rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    elif feature in zero_features:
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log) = probplot(np.log(orig + epsilon), rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig), rvalue=True)
        _, (*_, R_sqrt) = probplot(np.sqrt(orig), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig), rvalue=True)
        _, (*_, R_reci) = probplot(np.reciprocal(orig + epsilon), rvalue=True)
        _, (*_, R_boxcox) = probplot(stats.boxcox(orig + epsilon)[0], rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    elif feature == 'GDP':
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log) = probplot(np.log(orig + GDP_shift), rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig + GDP_shift), rvalue=True)
        _, (*_, R_sqrt) = probplot(np.sqrt(orig + GDP_shift), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig + GDP_shift), rvalue=True)
        _, (*_, R_reci) = probplot(np.reciprocal(orig + GDP_shift), rvalue=True)
        _, (*_, R_boxcox) = probplot(stats.boxcox(orig + GDP_shift)[0], rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)


    elif feature == 'Inflation Rate':
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log) = probplot(np.log(orig + Inflation_shift), rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig + Inflation_shift), rvalue=True)
        _, (*_, R_sqrt) = probplot(np.sqrt(orig + Inflation_shift), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig + Inflation_shift), rvalue=True)
        _, (*_, R_reci) = probplot(np.reciprocal(orig + Inflation_shift), rvalue=True)
        _, (*_, R_boxcox) = probplot(stats.boxcox(orig + Inflation_shift)[0], rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)

    r2_scores[feature] = (
        R_orig * R_orig,
        R_log * R_log,
        R_log1p * R_log1p,
        R_sqrt * R_sqrt,
        R_square * R_square,
        R_reci * R_reci,
        R_boxcox * R_boxcox,
        R_yeojohn * R_yeojohn
    )

r2_scores = pd.DataFrame(
    r2_scores, index=("Original", "Log", "Log1p", "Sqrt", "Square", "Reciprocal", "BoxCox", "YeoJohnson")
).T

r2_scores["HighestScore"] = r2_scores[["Original", "Log", "Log1p", "Sqrt", "Square", "Reciprocal", "BoxCox", "YeoJohnson"]].max(axis = 1)
r2_scores["Winner"] = r2_scores.idxmax(axis=1)


def highlight_max(s):
    is_max = s == s.max()
    return [f'background-color: {TEAL}' if v else '' for v in is_max]

r2_scores['Improvement'] = r2_scores['HighestScore'] - r2_scores['Original']
r2_scores.style.set_table_styles(DF_STYLE).apply(highlight_max, subset= ["Original", "Log", "Log1p", "Sqrt", "Square", "Reciprocal", "BoxCox", "YeoJohnson"], axis=1).background_gradient(cmap = DF_CMAP2, subset = 'Improvement').format(precision = 3)#.background_gradient(cmap=DF_CMAP, subset= ["Original", "Log", "log1p", "Sqrt", "Reciprocal", "BoxCox", "YeoJohnson"], axis = 1).format(precision=5)

,Original,Log,Log1p,Sqrt,Square,Reciprocal,BoxCox,YeoJohnson,HighestScore,Winner,Improvement
Marital status,0.276,0.313,0.305,0.299,0.207,0.312,0.312,0.312,0.313,Log,0.037
Application mode,0.779,0.739,0.752,0.782,0.709,0.634,0.742,0.751,0.782,Sqrt,0.004
Application order,0.593,0.635,0.628,0.622,0.500,0.571,0.598,0.603,0.635,Log,0.042
Course,0.356,0.227,0.227,0.281,0.517,0.000,0.752,0.752,0.752,YeoJohnson,0.396
Previous qualification,0.338,0.384,0.380,0.367,0.284,0.381,0.383,0.384,0.384,Log,0.046
Previous qualification (grade),0.980,0.979,0.979,0.981,0.970,0.967,0.981,0.981,0.981,YeoJohnson,0.001
Nacionality,0.039,0.046,0.045,0.044,0.022,0.047,0.047,0.047,0.047,BoxCox,0.008
Mother's qualification,0.793,0.739,0.749,0.782,0.756,0.548,0.778,0.782,0.793,Original,0.000
Father's qualification,0.776,0.678,0.695,0.746,0.758,0.492,0.765,0.770,0.776,Original,0.000
Mother's occupation,0.226,0.405,0.775,0.525,0.098,0.160,0.667,0.794,0.794,YeoJohnson,0.568


Looking at the table, log, while it does win for some features, wins by a small margin compared to log1p. We'll remove the log transformation to avoid overcomplicating the process. As for GDP and inflation rate, Yeo Johnson comes out as the best transformation by a small margin, and Sqrt the strongest for inflation rate. Looking at our improvement values however, the improvements are negligible.

Because of this I'll drop the sqrt transformation as its unable to handle the original values anyway. Reciprocal transformation can also be dropped. 

In the cases BoxCox wins, its value is identical to that of the YeoJohnson transform, and so I'll drop BoxCox too

Hence We're left with:
- Original
- log1p
- Square
- YeoJohnson

In [30]:
r2_scores = defaultdict(tuple)

for feature in numeric_cols:
    orig = train_sampled[feature].dropna()
    if feature in positive_features:
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig), rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    elif feature in zero_features:
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig), rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    elif feature == 'GDP':
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig + GDP_shift), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig + GDP_shift), rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    elif feature == 'Inflation Rate':
        _, (*_, R_orig) = probplot(orig, rvalue=True)
        _, (*_, R_log1p) = probplot(np.log1p(orig + Inflation_shift), rvalue=True)
        _, (*_, R_square) = probplot(np.square(orig + Inflation_shift), rvalue=True)
        _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)

    r2_scores[feature] = (
        R_orig * R_orig,
        R_log1p * R_log1p,
        R_square * R_square,
        R_yeojohn * R_yeojohn
    )

r2_scores = pd.DataFrame(
    r2_scores, index=("Original", "Log1p", "Square", "YeoJohnson")
).T

r2_scores["HighestScore"] = r2_scores[["Original", "Log1p", "Square", "YeoJohnson"]].max(axis = 1)
r2_scores["Winner"] = r2_scores.idxmax(axis=1)


def highlight_max(s):
    is_max = s == s.max()
    return [f'background-color: {TEAL}' if v else '' for v in is_max]

r2_scores['Improvement'] = r2_scores['HighestScore'] - r2_scores['Original']
r2_scores.style.set_table_styles(DF_STYLE).apply(highlight_max, subset= ["Original", "Log1p", "Square", "YeoJohnson"], axis=1).background_gradient(cmap = DF_CMAP2, subset = 'Improvement').format(precision = 3)

,Original,Log1p,Square,YeoJohnson,HighestScore,Winner,Improvement
Marital status,0.276,0.305,0.207,0.312,0.312,YeoJohnson,0.036
Application mode,0.779,0.752,0.709,0.751,0.779,Original,0.000
Application order,0.593,0.628,0.500,0.603,0.628,Log1p,0.035
Course,0.356,0.227,0.517,0.752,0.752,YeoJohnson,0.396
Previous qualification,0.338,0.380,0.284,0.384,0.384,YeoJohnson,0.046
Previous qualification (grade),0.980,0.979,0.970,0.981,0.981,YeoJohnson,0.001
Nacionality,0.039,0.045,0.022,0.047,0.047,YeoJohnson,0.008
Mother's qualification,0.793,0.749,0.756,0.782,0.793,Original,0.000
Father's qualification,0.776,0.695,0.758,0.770,0.776,Original,0.000
Mother's occupation,0.226,0.775,0.098,0.794,0.794,YeoJohnson,0.568


In [ ]:
Mother_occupation_orig = train_sampled["Mother\'s occupation"].dropna()
(osm, osr), (slope, intercept, R) = probplot(Mother_occupation_orig, rvalue=True)
x_theory = np.array([osm[0], osm[-1]])
y_theory = intercept + slope * x_theory

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=["Probability Plot against Normal Distribution", "Histogram"],
)

fig.add_scatter(x=osm, y=osr, mode="markers", row=1, col=1, name="AB")
fig.add_scatter(x=x_theory, y=y_theory, mode="lines", row=1, col=1)
fig.add_annotation(
    x=-1.25,
    y=osr[-1] * 0.4,
    text=f"R\u00b2 = {R * R:.3f}",
    showarrow=False,
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Observed Values", row=1, col=1)
fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=1)
fig.update_traces(
    marker=dict(size=3, symbol="circle", line=dict(width=2, color=DARK_TEAL)),
    line_color= ORANGE,
)

fig.add_histogram(
    x=Mother_occupation_orig,
    marker_color= DARK_TEAL,
    opacity=0.75,
    name="Mothers Occupation",
    row=1,
    col=2,
)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_xaxes(title_text="Mothers Occupation", row=1, col=2)

fig.update_layout(
    font_color=FONT_COLOR,
    title="Mothers Occupation Feature - Original",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    showlegend=False,
    width=1600,
    height=900,
    bargap=0.2,
)

fig.update_annotations(font_size=14)
fig.show()

In [ ]:
Mother_occupation_orig = stats.yeojohnson(train_sampled["Mother\'s occupation"].dropna())[0]
(osm, osr), (slope, intercept, R) = probplot(Mother_occupation_orig, rvalue=True)
x_theory = np.array([osm[0], osm[-1]])
y_theory = intercept + slope * x_theory

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=["Probability Plot against Normal Distribution", "Histogram"],
)

fig.add_scatter(x=osm, y=osr, mode="markers", row=1, col=1, name="Yeo Johnson(Mothers Occupation)")
fig.add_scatter(x=x_theory, y=y_theory, mode="lines", row=1, col=1)
fig.add_annotation(
    x=-1.25,
    y=osr[-1] * 0.6,
    text=f"R\u00b2 = {R * R:.3f}",
    showarrow=False,
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Observed Values", row=1, col=1)
fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=1)
fig.update_traces(
    marker=dict(size=3, symbol="circle", line=dict(width=2, color=DARK_TEAL)),
    line_color=ORANGE,
)

fig.add_histogram(
    x=Mother_occupation_orig,
    marker_color=DARK_TEAL,
    opacity=0.75,
    name="Mother\'s Occupation(Yeo Johnson)",
    row=1,
    col=2,
)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_xaxes(title_text="Yeo Johnson (Mothers Occupation)", row=1, col=2)

fig.update_layout(
    font_color=FONT_COLOR,
    title="Mothers Occupation Feature - Yeo Johnson Transformation",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    showlegend=False,
    width=1600,
    height=900,
    bargap=0.2,
)

fig.update_annotations(font_size=14)
fig.show()

In [32]:
r2_scores.describe().T.drop("count", axis=1).rename(
    columns=str.title
).style.set_table_styles(DF_STYLE).format(precision=3)

,Mean,Std,Min,25%,50%,75%,Max
Original,0.597,0.322,0.039,0.264,0.702,0.892,0.984
Log1p,0.587,0.280,0.045,0.361,0.678,0.766,0.991
Square,0.539,0.332,0.022,0.181,0.622,0.808,0.970
YeoJohnson,0.671,0.292,0.047,0.548,0.767,0.881,0.991
HighestScore,0.681,0.296,0.047,0.567,0.786,0.892,0.991
Improvement,0.083,0.159,0.000,0.001,0.013,0.046,0.568


In [33]:
no_transform_cols = r2_scores.query("Winner == 'Original'").index
log1p_transform_cols = r2_scores.query("Winner == 'Log1p'").index
square_transform_cols = r2_scores.query("Winner == 'Square'").index
yeojohnson_transform_cols = r2_scores.query("Winner == 'YeoJohnson'").index

In [35]:
initial_preprocess = make_pipeline(
    make_column_transformer(
        (
            StandardScaler(),
            no_transform_cols.to_list(),
        ),
        (
            make_pipeline(
                FunctionTransformer(func=np.log1p, feature_names_out="one-to-one"),
                StandardScaler(),
            ),
            log1p_transform_cols.to_list(),
        ),
        (
            make_pipeline(
                FunctionTransformer(func=np.square, feature_names_out="one-to-one"),
                StandardScaler(),
            ),
            square_transform_cols.to_list(),
        ),
        (
            PowerTransformer(method="yeo-johnson", standardize=True),
            yeojohnson_transform_cols.to_list(),
        ),
#        (
#            make_pipeline(
#                SimpleImputer(strategy="most_frequent"),
#                OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
#            ),
#            make_column_selector(dtype_include=object),  # type: ignore
        remainder="passthrough",
        verbose_feature_names_out=False,
        ),
    )

In [ ]:
initial_preprocess

With only 3 types of transformations, we're left with a short and simple transformation pipeline.

In [36]:
X = train.drop(target, axis=1)
y = train[target]

X_processed = initial_preprocess.fit_transform(X)
X_processed_frame = pd.DataFrame(
    X_processed,
    columns=initial_preprocess.get_feature_names_out(),
    index=X.index,
)
X_processed_frame.head().style.set_table_styles(DF_STYLE).format(precision=3)

,Application mode,Mother's qualification,Father's qualification,Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 2nd sem (approved),Application order,Admission grade,Curricular units 1st sem (grade),Curricular units 2nd sem (grade),Marital status,Course,Previous qualification,Previous qualification (grade),Nacionality,Mother's occupation,Father's occupation,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Daytime/evening attendance,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,International
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.902,-1.223,-0.297,0.065,-0.385,0.678,0.719,-0.573,-0.174,1.156,0.412,-0.300,-0.077,-0.372,-0.566,-0.000,-0.170,-0.283,-1.070,-0.205,-0.182,-0.192,-0.004,-0.084,-0.170,-0.078,-0.350,0.962,1.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000
1,0.057,-0.054,-0.297,0.065,0.185,-0.066,-1.445,-0.573,-0.406,0.097,-1.639,-0.300,-0.077,-0.372,-0.659,-0.000,0.608,0.524,-1.070,-0.205,-0.182,-0.192,-0.004,0.497,-0.170,-0.078,-0.350,0.962,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000
2,0.057,-1.093,-0.297,0.065,-2.096,-1.555,-1.445,0.567,1.489,-1.785,-1.639,-0.300,-0.051,-0.372,0.439,-0.000,-1.312,-0.953,-1.070,-0.205,-0.182,-0.192,-0.004,-2.018,-0.170,1.628,-0.587,-0.510,1.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000
3,-0.902,-0.054,-1.369,0.663,0.470,1.050,1.079,1.375,0.108,0.432,0.544,-0.300,0.375,-0.372,-0.103,-0.000,-0.825,-1.445,-1.070,-0.205,-0.182,-0.192,1.343,1.086,-0.170,-0.078,-0.350,0.962,1.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000
4,-0.902,-0.054,0.910,0.663,1.325,0.678,0.719,0.567,-0.381,0.554,0.582,-0.300,0.375,-0.372,-0.012,-0.000,-0.460,0.524,-1.070,-0.205,-0.182,-0.192,0.656,1.382,-0.170,-1.608,0.978,0.025,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000


In [ ]:
#make sure we still have all the data
X_processed_frame.shape

In [ ]:
X_processed.shape

In [ ]:
X_processed_sampled = X_processed[0:1000]
X_index_sampled = X[0:1000]

# T-SNE Time

In [ ]:
tsne_2D = TSNE(n_components=2, n_jobs=-1, random_state=42, perplexity=15, max_iter = 3000)

X_2D = pd.DataFrame(
    tsne_2D.fit_transform(X_processed_sampled), columns=["dim1", "dim2"], index=X_index_sampled.index
).join(y.astype(str))

In [ ]:
fig = px.scatter(
    X_2D.reset_index(),
    x="dim1",
    y="dim2",
    symbol="Target",
    symbol_sequence=["diamond", "circle", "square"],
    color="Target",
    color_discrete_sequence=[TEAL, ORANGE, DARK_TEAL],
    category_orders={"Target": ("0", "1", "2")},
    hover_data="id",
    opacity=0.7,
    height=840,
    width=840,
    title="Training Dataset - 2D Projection with t-SNE",
)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        xanchor="right",
        y=1.05,
        x=1,
        title="Target",
        itemsizing="constant",
    ),
)
fig.update_traces(marker_size=6)
fig.show()

In [ ]:
tsne_3D = TSNE(n_components=3, n_jobs=-1, random_state=42, perplexity=30)

X_3D = pd.DataFrame(
    tsne_3D.fit_transform(X_processed_sampled), columns=["dim1", "dim2", "dim3"], index=X_index_sampled.index
).join(y.astype(str))

In [ ]:
fig = px.scatter_3d(
    X_3D.reset_index(),
    x="dim1",
    y="dim2",
    z="dim3",
    symbol="Target",
    symbol_sequence=["diamond", "circle", "square"],
    color="Target",
    color_discrete_sequence=[TEAL, ORANGE, DARK_TEAL],
    category_orders={"Target": ("0", "1", "2")},
    hover_data="id",
    opacity=0.7,
    height=840,
    width=840,
    title="Training Dataset - 3D Projection with t-SNE",
)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        xanchor="right",
        y=1.05,
        x=1,
        title="Target",
        itemsizing="constant",
    ),
)
fig.update_traces(marker_size=3)
fig.show()

# Feature Importance - Embedded Methods - Random Forest Impurity Based Feature Importance

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, train_size = 0.7)

In [50]:
#lda_pipeline = make_pipeline(
#    initial_preprocess,
#    LinearDiscriminantAnalysis(n_components= 2),
#).fit(X, y)
#
#lda_scalings = np.abs(lda_pipeline[-1].scalings_)
#lda_info = np.mean(lda_scalings, axis=1) 
#lda_info = lda_info / lda_info.sum()  # Normalize to 1 to compare with other methods




cat_pipeline = make_pipeline(
    initial_preprocess,
    CatBoostClassifier(random_seed = 42, auto_class_weights= "Balanced", classes_count= 3, thread_count= -1, max_depth= 5, n_estimators= 100)
).fit(X_train,y_train)

cat_pipeline.predict(X_test)

cat_info = cat_pipeline[-1].feature_importances_
cat_info = cat_info / cat_info.sum()



lgbm_pipeline = make_pipeline(
    initial_preprocess,
    LGBMClassifier(random_state=42, class_weight = 'balanced', n_jobs = -1, objective = 'multiclass', max_depth = 5, n_estimators= 500, verbose = 100),
).fit(X_train,y_train)

print(f"lgbm Accuracy: {accuracy_score(y_test, lgbm_pipeline.predict(X_test))}")



lgbm_info = lgbm_pipeline[-1].feature_importances_
lgbm_info = lgbm_info / lgbm_info.sum()



xgb_pipeline = make_pipeline(
    initial_preprocess,
    XGBClassifier(random_state=42, class_weight = 'balanced', n_jobs = -1, objective = 'multiclass', max_depth = 10, n_estimators= 200, verbose = 100),
).fit(X_train,y_train)

lgbm_pipeline.predict(X_test)


xgb_info = xgb_pipeline[-1].feature_importances_
xgb_info = xgb_info / xgb_info.sum()







mutual_info = mutual_info_classif(
    X=initial_preprocess.fit_transform(X), y=y, random_state=42
)
mutual_info = mutual_info / np.sum(mutual_info)

importances = pd.DataFrame(
    [cat_info, lgbm_info, xgb_info, mutual_info],
    columns=lgbm_pipeline[0].get_feature_names_out(),
    index=["CAT","LGBM", "XGB", "MI"],
).T

importances.style.set_table_styles(DF_STYLE).format(precision=4)


Learning rate set to 0.5
0:	learn: 0.7467882	total: 17.6ms	remaining: 1.74s
1:	learn: 0.6461758	total: 34.8ms	remaining: 1.7s
2:	learn: 0.5952304	total: 51.4ms	remaining: 1.66s
3:	learn: 0.5723680	total: 67.8ms	remaining: 1.63s
4:	learn: 0.5579137	total: 85.5ms	remaining: 1.62s
5:	learn: 0.5488741	total: 101ms	remaining: 1.59s
6:	learn: 0.5410763	total: 119ms	remaining: 1.57s
7:	learn: 0.5362234	total: 135ms	remaining: 1.55s
8:	learn: 0.5325808	total: 148ms	remaining: 1.5s
9:	learn: 0.5292192	total: 158ms	remaining: 1.42s
10:	learn: 0.5271053	total: 168ms	remaining: 1.36s
11:	learn: 0.5249717	total: 180ms	remaining: 1.32s
12:	learn: 0.5219247	total: 193ms	remaining: 1.29s
13:	learn: 0.5205301	total: 204ms	remaining: 1.25s
14:	learn: 0.5190985	total: 219ms	remaining: 1.24s
15:	learn: 0.5179301	total: 235ms	remaining: 1.23s
16:	learn: 0.5174137	total: 250ms	remaining: 1.22s
17:	learn: 0.5162916	total: 272ms	remaining: 1.24s
18:	learn: 0.5152230	total: 298ms	remaining: 1.27s
19:	learn: 0.

,CAT,LGBM,XGB,MI
Application mode,0.0129,0.0235,0.0089,0.0266
Mother's qualification,0.0074,0.0277,0.0080,0.0100
Father's qualification,0.0074,0.0293,0.0080,0.0100
Curricular units 1st sem (enrolled),0.0123,0.0161,0.0169,0.0237
Curricular units 1st sem (evaluations),0.0317,0.0442,0.0108,0.0564
Curricular units 1st sem (approved),0.0628,0.0309,0.0163,0.1248
Curricular units 2nd sem (approved),0.2051,0.0405,0.3544,0.1473
Application order,0.0047,0.0172,0.0089,0.0038
Admission grade,0.0268,0.1051,0.0085,0.0322
Curricular units 1st sem (grade),0.0247,0.0876,0.0090,0.1128


In [51]:
importance_score_summary = importances.T.describe().T

importance_score_summary[['min', 'max', 'mean']].sort_values(by = 'mean', ascending = False).style.set_table_styles(DF_STYLE).background_gradient(
                                                                        cmap = DF_CMAP2, 
                                                                        subset = ['mean'], 
                                                                        vmin = importance_score_summary['mean'].min(),
                                                                        vmax = importance_score_summary['mean'].mean(),
                                                            ).background_gradient(
                                                                        cmap = DF_CMAP, 
                                                                        subset = 'min', 
                                                                        vmin = importance_score_summary['min'].min(),
                                                                        vmax = importance_score_summary['min'].mean(),
                                                            ).background_gradient(
                                                                        cmap = DF_CMAP, 
                                                                        subset = 'max', 
                                                                        vmin = importance_score_summary['max'].min(),
                                                                        vmax = importance_score_summary['max'].mean(),
                                                                        
                                                            ).format(precision = 3)

,min,max,mean
Curricular units 2nd sem (approved),0.040,0.354,0.187
Curricular units 2nd sem (grade),0.014,0.216,0.112
Tuition fees up to date,0.011,0.233,0.088
Curricular units 1st sem (approved),0.016,0.125,0.059
Curricular units 1st sem (grade),0.009,0.113,0.059
Curricular units 2nd sem (evaluations),0.021,0.060,0.044
Admission grade,0.008,0.105,0.043
Course,0.010,0.054,0.037
Curricular units 1st sem (evaluations),0.011,0.056,0.036
Previous qualification (grade),0.008,0.072,0.031


In [52]:
importances_melted_frame = (
    importances.melt(
        var_name="Method",
        value_name="Importance",
        ignore_index=False,
    )
    .reset_index()
    .rename(columns={"index": "Feature"})
    .round(4)
)

fig = px.bar(
    importances_melted_frame,
    x="Importance",
    y="Feature",
    color="Importance",
    facet_col="Method",
    facet_col_spacing=0.07,
    height=2000,
    width=2000,
    color_continuous_scale=color_map,
    title="Normalised Feature Importances (Three Different Default Methods)",
)
fig.update_annotations(font_size=14)
fig.update_yaxes(
    matches=None,
    showticklabels=True,
    categoryorder="total ascending",
    tickfont_size=8,
)
fig.update_xaxes(matches=None)
fig.update_traces(width=0.7)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    coloraxis_colorbar=dict(
        orientation="h",
        title_side="bottom",
        yanchor="bottom",
        xanchor="center",
        title=None,
        y=-0.2,
        x=0.5,
    ),
)
fig.show()

# Feature Importance - Embedded Methods - Sample Permutation Feature Importance

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, brier_score_loss, average_precision_score
import numpy as np
import pandas as pd
from collections import defaultdict
from functools import partial
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

# Define the metric function to be used. Note: We need the probability values of each class, not prediction values, hence we can't use metrics like accuracy. 
def evaluation_metric(y_true, y_pred_proba, metric='log loss'):
    """Evaluate the chosen metric."""
    if metric == 'log loss':
        return log_loss(y_true, y_pred_proba)
    elif metric == 'roc auc':
        return roc_auc_score(y_true, y_pred_proba, multi_class='ovr')
    elif metric == 'brier':
        return brier_score_loss(y_true, y_pred_proba[:, 1])
    elif metric == 'average precision':
        return average_precision_score(y_true, y_pred_proba, average='micro')
    else:
        raise ValueError("Unsupported metric provided.")

n_bags = 5
n_folds = 5
metric = 'log loss'  # Change this to 'roc_auc', 'brier', or 'average_precision' as needed

np.random.seed(42)
seeds = np.random.randint(0, 19937, size=n_bags)
original_scores = []
permutation_scores = pd.DataFrame()

forest = RandomForestClassifier(class_weight="balanced", random_state=42, n_jobs = -1)
cat = CatBoostClassifier(random_state = 42, auto_class_weights= "Balanced", classes_count= 3, thread_count= -1)
lgbm = LGBMClassifier(random_state=42, class_weight = 'balanced', n_jobs = -1, objective = 'multiclass', max_depth = 15, n_estimators= 1000, verbose = 10)


classifiers = ['rf', 'cat', 'lgbm']

for selected_classifier in classifiers:
    y_proba_original = np.zeros((len(y), len(np.unique(y))), dtype=np.float64)
    y_proba_shuffled = defaultdict(partial(np.zeros, (len(y), len(np.unique(y))), dtype=np.float64))



    for seed in seeds:
        skfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

        if selected_classifier == 'rf':
            classifier = RandomForestClassifier(class_weight="balanced", random_state = seed, n_jobs = -1)
        elif selected_classifier == 'cat':
            classifier = CatBoostClassifier(random_state = seed, auto_class_weights= "Balanced", classes_count= 3, thread_count= -1)
        elif selected_classifier == 'lgbm':
            classifier = LGBMClassifier(random_state = seed, class_weight = 'balanced', n_jobs = -1, objective = 'multiclass', max_depth = 15, n_estimators= 1000, verbose = 10)




        for train_ids, valid_ids in skfold.split(X, y):
            X_train, y_train = X.iloc[train_ids], y.iloc[train_ids]
            X_valid, y_valid = X.iloc[valid_ids], y.iloc[valid_ids]

            X_train = initial_preprocess.fit_transform(X_train)
            X_valid = initial_preprocess.transform(X_valid)

            classifier.fit(X_train, y_train)
            y_proba_original[valid_ids] += classifier.predict_proba(X_valid)

            for i, feature in enumerate(initial_preprocess.get_feature_names_out()):
                X_shuffled = X_valid.copy()
                X_shuffled[:, i] = np.random.permutation(X_shuffled[:, i])  # type: ignore
                y_proba_shuffled[feature][valid_ids] += classifier.predict_proba(X_shuffled)

    classifier_name = classifier.__class__.__name__
    feature_names = y_proba_shuffled.keys()
    print("step 1")
    original_scores.append(evaluation_metric(y, y_proba_original / n_bags, metric))
    print("step 2")
    permutation_scores[classifier_name] = pd.Series(
        [
            evaluation_metric(y, y_proba_shuffled[feature] / n_bags, metric)
            for feature in feature_names
        ],
        index=feature_names,
    )

# Display the results
print("Original scores:", original_scores)
print("Permutation scores:\n", permutation_scores)

In [ ]:
permutation_score_summary = permutation_scores.T.describe().T

permutation_score_summary[['min', 'max', 'mean']].sort_values(by = 'mean', ascending = False).style.set_table_styles(DF_STYLE).background_gradient(
                                                                        cmap = DF_CMAP2, 
                                                                        subset = ['mean'], 
                                                                        vmin = permutation_score_summary['mean'].min(),
                                                                        vmax = permutation_score_summary['mean'].mean(),
                                                            ).background_gradient(
                                                                        cmap = DF_CMAP, 
                                                                        subset = 'min', 
                                                                        vmin = permutation_score_summary['min'].min(),
                                                                        vmax = permutation_score_summary['min'].mean(),
                                                            ).background_gradient(
                                                                        cmap = DF_CMAP, 
                                                                        subset = 'max', 
                                                                        vmin = permutation_score_summary['max'].min(),
                                                                        vmax = permutation_score_summary['max'].mean(),
                                                                        
                                                            ).format(precision = 3)

In [ ]:
permutation_results_melted = (
    permutation_scores.melt(
        var_name="Method",
        value_name=f"{metric}",
        ignore_index=False,
    )
    .reset_index()
    .rename(columns={"index": "Feature"})
    .round(4)
)

fig = px.bar(
    permutation_results_melted,
    x=f"{metric}",
    y="Feature",
    color=f"{metric}",
    facet_col="Method",
    facet_col_spacing=0.07,
    height=2000,
    width=2000,
    color_continuous_scale=color_map,
    title=f"Permutation Test Results - {metric} when Permuting Samples<br>"
    f"in Certain Features (Averaged over Stratified {n_folds}-Fold and {n_bags} Different Seeds)",
)
fig.update_annotations(font_size=14)
fig.update_traces(width=0.7)
fig.update_xaxes(matches=None)
fig.update_yaxes(
    matches=None,
    showticklabels=True,
    categoryorder="total ascending",
    tickfont_size=8,
)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    coloraxis_colorbar=dict(
        orientation="h",
        title_side="bottom",
        yanchor="bottom",
        xanchor="center",
        title=None,
        y=-0.2,
        x=0.5,
    ),
    margin_t=120,
)
for original_score, max_score, col in zip(
    original_scores, permutation_scores.max().tolist(), (1, 2, 3)
):
    fig.add_vline(
        x=original_score,
        line_width=2,
        line_dash="dash",
        line_color= DARK_TEAL,
        col=col,
    )
    fig.add_vrect(
        x0=original_score,
        x1=max_score,
        line_width=0,
        fillcolor=DARK_TEAL,
        opacity=0.2,
        col=col,
    )
fig.show()

# Feature Importance - Wrapper Methods - Sequential Feature Selection

In [37]:

#sfs_forward = SFS(model, k_features = (3, 10), forward = True, floating = False, verbose = 2, scoring = 'accuracy', cv = 5, n_jobs = -1)
#sfs_forward.fit(X_train, y_train)
#sfs_forward.k_feature_idx
#
#
#from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
#metric_dict = sfs_forward.get_metric_dict(confidence_interval = 0.95)
#fig = plot_sfs(metric_dict, kind = 'std_dev)
#plt.show()
#
#
#After finding the best feature subset, we can use:
#X_train_selected_features = ssfs_forward.transform(X_train)
#X_test_selected_features = ssfs_forward.transform(X_test)


# Initialize Sequential Floating Backward Selection
sfs_backward = SFS(estimator = LGBMClassifier(random_state=42, 
                                              class_weight = 'balanced', 
                                              n_jobs = -1, 
                                              objective = 'multiclass'),
    k_features=(3, 6),
    forward=True,          # Set forward to False for backward selection
    floating=True,          # Set floating to True for floating selection
    verbose=2,
    scoring='f1_micro',
    cv=4,
)

sfs_backward_pipeline = make_pipeline(
    initial_preprocess,
    sfs_backward
).fit(X,y)

selected_feature_indices = sfs_backward.k_feature_idx_
selected_feature_names = initial_preprocess.get_feature_names_out()
selected_features = [selected_feature_names[i] for i in selected_feature_indices]
selected_features

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 57388, number of used features: 1
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

TypeError: 'NoneType' object is not iterable

In [ ]:
# Print the selected feature indices
print("Selected feature indices:", sfs_backward.k_feature_idx_)

# Plot the sequential feature selection process
metric_dict = sfs_backward.get_metric_dict(confidence_interval=0.95)
fig = plot_sfs(metric_dict, kind='std_dev')

sfs_backward_pipeline.__dict__
plt.title('Sequential Floating Backward Selection (SFBS)')
plt.grid()
plt.show()
#
## Transform the training and testing data to the selected features
X_train_selected_features = sfs_backward.transform(X)
#X_test_selected_features = sfs_backward_pipeline.transform(X_test)
#
## Optionally, you can now train a model with the selected features
#model.fit(X_train_selected_features, y_train)
#accuracy = model.score(X_test_selected_features, y_test)
#print(f"Accuracy with selected features: {accuracy:.4f}")
X_train_selected_features

# Model Selection Using Pycaret

# PROBABLY REMOVE THE VOTING CLASSIFIER FROM THE PIPELINE, SEEMS IT COUDLD BE BETTER TO DO IT AFTER SO YOU CAN ADJUST THE VALUES ON AN ALREADY TRAINED SET OF MODELS

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from collections import defaultdict

n_bags = 1
n_folds = 4

np.random.seed(42)
seeds = np.random.randint(0, 19937, size=n_bags)

X = train.drop("Target", axis=1)[0:5000]  # Adjust this to your actual feature set
y = train['Target'][0:5000]  # Adjust this to your actual target column

# Define your parameter grids for each classifier
#lgbm_params = {
  #  "max_depth": [1,2,3,4,5,6,7,8,9,10],
    #"num_leaves": [6, 12, 18],
  #  "n_estimators": [100, 200, 300],
    #"learning_rate": [0.1, 0.15, 0.2],
#}

#xgb_params = {
#    "max_depth": [2, 4, 6],
#    "n_estimators": [100, 200, 300],
#    "learning_rate": [0.2, 0.3, 0.4],
#}
#
#svc_params = {
 #   "C": [1, 3, 5],
#}


param_distributions = {
            "votingclassifier__lgbm__max_depth": [7,10,11,12],
            "votingclassifier__lgbm__n_estimators": [50, 100, 300, 500],
            #"lgbmclassifier__num_leaves": lgbm_params["num_leaves"],
            #"lgbmclassifier__n_estimators": lgbm_params["n_estimators"],
            #"lgbmclassifier__learning_rate": lgbm_params["learning_rate"]
            #"votingclassifier__xgb__max_depth": [6,7,10,11],
            #"xgb__classifier__n_estimators": xgb_params["n_estimators"],
            #"xgb__classifier__learning_rate": xgb_params["learning_rate"],
            "votingclassifier__svc__C": [0.5,0.8,1,1.2],
        }


classifiers = defaultdict(object)
fold_accuracies = []
val_results_data = []#pd.DataFrame(columns=["Bag", "Fold", "Best Parameters", "Validation Accuracy"])

for bag, seed in enumerate(seeds):
    skfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    print(f"Beginning Bag {bag}")
    for fold, (train_ids, valid_ids) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_ids], y.iloc[train_ids]
        X_valid, y_valid = X.iloc[valid_ids], y.iloc[valid_ids]
        print(f"fold {fold}")
        current_ensemble = make_pipeline(
            initial_preprocess,  # Example preprocessing step (replace with your own)
            VotingClassifier(
                [
                    ("lgbm", LGBMClassifier(random_state=seed, class_weight = 'balanced', n_jobs = -1, objective = 'multiclass')),
                    ("xgb", XGBClassifier(random_state=seed, n_jobs = -1)),
                    #("cat", CatBoostClassifier(random_state = seed, auto_class_weights= "Balanced", classes_count= 3, thread_count= -1, verbose = 10)),
                    ("svc", SVC(random_state=seed, class_weight = 'balanced'))
                ],  
                voting="hard",
                #weights=(0.25, 0.25, 0.5),
            ),
        )

        # Define RandomizedSearchCV for parameter tuning

        #param_distributions = {
          #  "votingclassifier__lgbm__max_depth": [0,1,2,3,4,5,6,7,8,9,10],
            #"lgbmclassifier__num_leaves": lgbm_params["num_leaves"],
            #"lgbmclassifier__n_estimators": lgbm_params["n_estimators"],
            #"lgbmclassifier__learning_rate": lgbm_params["learning_rate"]
            #"xgb__classifier__max_depth": xgb_params["max_depth"],
            #"xgb__classifier__n_estimators": xgb_params["n_estimators"],
            #"xgb__classifier__learning_rate": xgb_params["learning_rate"],
            #"svc__classifier__C": svc_params["C"],
       # }



        random_search = RandomizedSearchCV(
            estimator=current_ensemble,
            param_distributions=param_distributions,
            n_iter=4,  # Adjust as needed
            scoring="accuracy",
            cv=skfold,
            random_state=seed,
            n_jobs=-1,
            return_train_score = True,
            verbose = 3
        )

        random_search.fit(X_train, y_train)
        best_estimator = random_search.best_estimator_
        classifiers[f"Voting Bag: {bag} Fold: {fold}"] = best_estimator

        #for est_name, est in best_estimator.named_steps['votingclassifier'].estimators:
        #  if est_name == 'cat':
        #    y_pred_valid = y_pred_valid.ravel()

        #y_pred_valid_lgbm = best_estimator.named_steps['votingclassifier'].estimators_[0].predict(X_valid)
        #fold_accuracy_lgbm = accuracy_score(y_valid, y_pred_valid_lgbm)
        #print(f"lgbm fold accuracy: {fold_accuracy_lgbm}")
#
#
        #y_pred_valid_xgb = best_estimator.named_steps['votingclassifier'].estimators_[1].predict(X_valid)
        #fold_accuracy_xbg = accuracy_score(y_valid, y_pred_valid_xgb)
        #print(fold_accuracy_xbg)
#
        #y_pred_valid_svc = best_estimator.named_steps['votingclassifier'].estimators_[2].predict(X_valid)
        #fold_accuracy_svc = accuracy_score(y_valid, y_pred_valid_svc)
        #print(fold_accuracy_svc)


        #y_pred_valid_cat = best_estimator.named_steps['votingclassifier'].estimators_[2].predict(X_valid)
        #fold_accuracy_cat = 
        #print(cat_pred)



        # Evaluate on validation set
        y_pred_valid = best_estimator.predict(X_valid)
        fold_accuracy = accuracy_score(y_valid, y_pred_valid)
        fold_accuracies.append(fold_accuracy)
        print(f"Bag: {bag}, Fold: {fold} - Validation Accuracy: {fold_accuracy:.4f}")



        results = [bag, fold, random_search.best_params_, fold_accuracy]
        val_results_data.append(results)

        #results_df = results_df.append({
        #    "Bag": bag,
        #    "Fold": fold,
        #    "Best Parameters": random_search.best_params_,
        #    "Validation Accuracy": fold_accuracy
        #}, ignore_index=True)

# After training and validation, you might want to test on a separate test set
# Example: Predict classes and calculate accuracy on a hold-out set
# Replace `X_test` and `y_test` with your actual test data

X_test = test
y_pred_test = best_estimator.predict(X_test)
#test_accuracy = accuracy_score(y_test, y_pred_test)
#print(f"Accuracy on test set: {test_accuracy:.4f}")

# Optionally, you can also compute mean accuracy across all folds
mean_accuracy = np.mean(fold_accuracies)
print(f"Mean Cross-Validation Accuracy: {mean_accuracy:.4f}")

In [ ]:
pd.set_option('display.max_colwidth', 500)
results_dataframe = pd.DataFrame(val_results_data).rename(columns = {0:'bag', 1: 'fold', 2:'best_parameters', 3:'validation_accuracy'})
results_dataframe

In [ ]:
test_ids = test.index

submission_data = pd.DataFrame({"id": test_ids,
              "Target": y_pred_test}).set_index('id')


target_dict = {
    2: 'Enrolled',
    0: 'Dropout',
    1: 'Graduate'
}


# Replace the values in the "Target" column
submission_data['Target'] = submission_data['Target'].replace(target_dict)

submission_data.to_csv('academic-success-predictions.csv')

In [ ]:
current_ensemble

In [ ]:
model1 = make_pipeline(LGBMClassifier())
model2 = make_pipeline(VotingClassifier([
                    ("lgbm", LGBMClassifier(random_state=seed)),
                    ("svc", SVC(random_state=seed)),
                    ("xgb", LGBMClassifier(random_state=seed)),
                    ]))

In [ ]:
[(k, v) for k, v in model2.get_params().items() if "lgbm" in k or "svc" in k or "xgb" in k]